In [1]:
# rdt model coreml

In [3]:
import tensorflow as tf
import numpy as np
import coremltools

print(tf.__version__)
print(coremltools.__version__)

2.1.0
3.2


In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(4)
        self.dense2 = tf.keras.layers.Dense(5)

    @tf.function
    def call(self, input_data):
        return self.dense2(self.dense1(input_data))

keras_model = MyModel()

In [5]:
inputs = np.random.rand(4, 4)

# subclassed model can only be saved as SavedModel format
keras_model._set_inputs(inputs)
saved_model_dir = './tf_model_subclassing'
keras_model.save(saved_model_dir, save_format='tf')
# convert and validate
model = coremltools.converters.tensorflow.convert(
    saved_model_dir,
    inputs={'input_1': (4, 4)},
    outputs=['Identity']
)
assert isinstance(model, coremltools.models.MLModel)
# verify the prediction matches
keras_prediction = keras_model.predict(inputs)
prediction = model.predict({'input_1': inputs})['Identity']
np.testing.assert_array_equal(keras_prediction.shape, prediction.shape)
np.testing.assert_almost_equal(keras_prediction.flatten(), prediction.flatten(), decimal=4)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./tf_model_subclassing/assets


INFO:tensorflow:Assets written to: ./tf_model_subclassing/assets


0 assert nodes deleted
16 nodes deleted
0 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 4 nodes.
2 identity nodes deleted
2 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/3] Converting op type: 'Placeholder', name: 'input_1', output_shape: (4, 4).
[SSAConverter] [2/3] Converting op type: 'MatMul', name: 'my_model/StatefulPartitionedCall/StatefulPartitionedCall/dense/StatefulPartitionedCall/MatMul', output_shape: (4, 4).
[SSAConverter] [3/3] Converting op type: 'MatMul', name: 'Identity', output_shape: (4, 5).
